In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType, Row
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, when, lit

In [2]:
spark = SparkContext(master="local", appName="DataFrame")
sqlContext = SQLContext(spark)

21/09/26 19:46:47 WARN Utils: Your hostname, DESKTOP-NVKHLIO resolves to a loopback address: 127.0.1.1; using 192.168.210.162 instead (on interface eth0)
21/09/26 19:46:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/09/26 19:46:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [14]:
spark.stop()

In [3]:
!head -n 5 ./files/deportistaError.csv

deportista_id,nombre,genero,edad,altura,peso,equipo_id
1,A Dijiang,1,24,180,80,199
2,A Lamusi,1,23,170,60,199
3,Gunnar Nielsen Aaby,1,24,,,273
4,Edgar Lindenau Aabye,1,34,,,278


In [4]:
deportistaError = spark.textFile("./files/deportistaError.csv")\
    .map(lambda line: line.split(","))

In [5]:
def eliminaEncabezado(indice,iterador):
    return iter(list(iterador)[1:])

In [6]:
deportistaError = deportistaError.mapPartitionsWithIndex(eliminaEncabezado)

In [7]:
deportistaError.take(3)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '', '', '273']]

In [10]:
deportistaError.map(lambda line: (
    line[0],
    line[1],
    line[2],
    line[3],
    line[4],
    line[5],
    line[6]))

schema = StructType([
    StructField("deportista_id", StringType(), False),
    StructField("nombre", StringType(), False),
    StructField("genero", StringType(), False),
    StructField("edad", StringType(), False),
    StructField("altura", StringType(), False),
    StructField("peso", StringType(), False),
    StructField("equipo_id", StringType(), False)
])

deportistaErrorDF = sqlContext.createDataFrame(deportistaError, schema)
    

In [11]:
deportistaErrorDF.show()

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|  80|      199|
|            2|            A Lamusi|     1|  23|   170|  60|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|      |    |      273|
|            4|Edgar Lindenau Aabye|     1|  34|      |    |      278|
|            5|Christine Jacoba ...|     2|  21|   185|  82|      705|
|            6|     Per Knut Aaland|     1|  31|   188|  75|     1096|
|            7|        John Aalberg|     1|  31|   183|  72|     1096|
|            8|"Cornelia ""Cor""...|     2|  18|   168|    |      705|
|            9|    Antti Sami Aalto|     1|  26|   186|  96|      350|
|           10|"Einar Ferdinand ...|     1|  26|      |    |      350|
|           11|  Jorma Ilmari Aalto|     1|  22|   182|76.5|      350|
|     

In [16]:
from pyspark.sql.functions import udf

def conversionEnteros(valor):
    try:
        return int(valor)
    except:
        return None

conversionEnterosUDF = udf(lambda z: conversionEnteros(z), IntegerType())
sqlContext.udf.register("conversionEnterosUDF", conversionEnterosUDF)

21/09/26 19:57:02 WARN SimpleFunctionRegistry: The function conversionenterosudf replaced a previously registered function.


<function __main__.<lambda>(z)>

In [17]:
deportistaErrorDF.select(conversionEnterosUDF("altura").alias("altura_int")).show()

21/09/26 19:57:05 ERROR PythonUDFRunner: Python worker exited unexpectedly (crashed)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/dsandovalflavio/spark/python/lib/pyspark.zip/pyspark/worker.py", line 584, in main
    eval_type = read_int(infile)
  File "/home/dsandovalflavio/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:517)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:84)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:67)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:470)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:489)
	at scala.collection

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/home/dsandovalflavio/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/home/dsandovalflavio/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/dsandovalflavio/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/dsandovalflavio/spark/python/lib/pyspark.zip/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/home/dsandovalflavio/spark/python/pyspark/sql/session.py", line 682, in prepare
    verify_func(obj)
  File "/home/dsandovalflavio/spark/python/pyspark/sql/types.py", line 1409, in verify
    verify_value(obj)
  File "/home/dsandovalflavio/spark/python/pyspark/sql/types.py", line 1388, in verify_struct
    "len